<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/lessons3/TranslateEng2Rus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Перевод текста на с испольванием эмбендинга word2vec fasttext

In [1]:
!pip install -q --upgrade nltk gensim bokeh pandas
!pip install wget

     |████████████████████████████████| 1.5 MB 6.9 MB/s 
     |████████████████████████████████| 24.1 MB 65.6 MB/s 
     |████████████████████████████████| 18.5 MB 1.4 MB/s 
     |████████████████████████████████| 749 kB 43.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.12.1 requires bokeh<2.4.0,>=2.3.0, but you have bokeh 2.4.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1208d394f5f011f093b60545be74081d0f2583a9ccd09e320eeb198daff9a318
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, KeyedVectors
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from string import punctuation
import wget

from google.colab import drive 
drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/gdrive


In [3]:
wget.download('https://dl.fbaipublicfiles.com/arrival/dictionaries/en-ru.0-5000.txt', 'en_ru.train.txt')
wget.download('https://dl.fbaipublicfiles.com/arrival/dictionaries/en-ru.5000-6500.txt', 'en_ru.test.txt')

wget.download('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz', 'cc.en.300.vec.gz')
wget.download('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz', 'cc.ru.300.vec.gz')

'cc.ru.300.vec.gz'

In [4]:
ru_vec = KeyedVectors.load_word2vec_format("/content/cc.ru.300.vec.gz")
en_vec = KeyedVectors.load_word2vec_format("/content/cc.en.300.vec.gz")


In [5]:
ru_vec.most_similar([ru_vec['кот']])

[('кот', 0.9999998807907104),
 ('котик', 0.7908911108970642),
 ('пес', 0.7660022377967834),
 ('Кот', 0.7640087008476257),
 ('котенок', 0.757697343826294),
 ('котяра', 0.7543642520904541),
 ('песик', 0.7248488664627075),
 ('пёс', 0.7238960266113281),
 ('котёнок', 0.7153405547142029),
 ('кошак', 0.7141522765159607)]

In [6]:
en_vec.most_similar([en_vec['cat']])

[('cat', 1.0),
 ('cats', 0.8350197672843933),
 ('kitty', 0.8233399391174316),
 ('kitten', 0.8082815408706665),
 ('feline', 0.7533785104751587),
 ('moggie', 0.7111802101135254),
 ('cat.It', 0.7107158303260803),
 ('dog', 0.7078642249107361),
 ('cat.The', 0.7065413594245911),
 ('cat.I', 0.7062530517578125)]

In [7]:
def load_word_pairs(filename):
    en_ru_pairs = []
    en_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            en, ru = line.rstrip().split(" ")
            if en not in en_vec or ru not in ru_vec:
                continue
            en_ru_pairs.append((en, ru))
            en_vectors.append(en_vec[en])
            ru_vectors.append(ru_vec[ru])
    return en_ru_pairs, np.array(en_vectors), np.array(ru_vectors)

In [8]:
en_ru_train, X_train, Y_train = load_word_pairs("en_ru.train.txt")
en_ru_test, X_test, Y_test = load_word_pairs("en_ru.test.txt")

In [9]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [10]:
august = mapping.predict(en_vec["august"].reshape(1, -1))
ru_vec.most_similar(august)

[('апреля', 0.6513441801071167),
 ('марта', 0.6481823921203613),
 ('июля', 0.6434776186943054),
 ('ноября', 0.642888605594635),
 ('июня', 0.6415560245513916),
 ('октябрь', 0.6412094235420227),
 ('октября', 0.6398786306381226),
 ('сентября', 0.6378851532936096),
 ('августа', 0.6375009417533875),
 ('сентябрь', 0.637001097202301)]

In [11]:
def precision(pairs, mapped_vectors, emb, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, dst) in enumerate(pairs):
        similar = emb.most_similar([mapped_vectors[i]],topn=topn)
        for word, p in similar:
            if dst == word:
                num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [12]:
def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T, Y_train))
    W = np.matmul(U, V)
    return W

In [13]:
W = learn_transform(X_train, Y_train)

In [14]:
print(precision(en_ru_test, np.matmul(X_test, W), ru_vec))

0.2865291794646775


In [15]:
with open("/content/gdrive/MyDrive/Colab Notebooks/nlp/Lessons 3 - Embedding word2vec fasttext/fairy_tales.txt", "r") as f:
    en_sentences = [line.rstrip().lower() for line in f]

In [26]:
def translate(sentence):
    words = sentence.split(" ")
    translation = []
    for word in words:
        # try:
            emb = en_vec[word]
            translation.append(ru_vec.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        # except:
        #     translation.append(word)
    return " ".join(translation)


In [29]:
# Ну, почти нормльно xD
translate("Yes i am work")


'Да ,,я я работу'